In [1]:
from pynucastro.screening import chugunov_2009
from keras_network import ScreeningFactorData, ScreeningFactorNetwork

import pynucastro as pyna
import numpy as np

In [2]:
data = ScreeningFactorData(screen_func=chugunov_2009, threshold=1.01, size=10**6, seed=1)

In [3]:
1 - data.frac_pos

0.003618333333333279

In [4]:
network = ScreeningFactorNetwork(data=data, seed=1)

In [5]:
network.compile()
history = network.fit_model(verbose=2)
network.score

Epoch 1/100
489/489 - 7s - 14ms/step - Brier score: 0.1084 - accuracy: 0.8445 - auc: 0.7968 - cross entropy: 0.3399 - fn: 154048.0000 - fp: 1457.0000 - loss: 0.6895 - prc: 0.9989 - precision: 0.9983 - recall: 0.8454 - tn: 2106.0000 - tp: 842389.0000 - val_Brier score: 0.1216 - val_accuracy: 0.8467 - val_auc: 0.9155 - val_cross entropy: 0.3771 - val_fn: 152412.0000 - val_fp: 889.0000 - val_loss: 0.3770 - val_prc: 0.9996 - val_precision: 0.9989 - val_recall: 0.8470 - val_tn: 2778.0000 - val_tp: 843921.0000
Epoch 2/100
489/489 - 3s - 5ms/step - Brier score: 0.1386 - accuracy: 0.7955 - auc: 0.8773 - cross entropy: 0.4222 - fn: 203629.0000 - fp: 852.0000 - loss: 0.4276 - prc: 0.9995 - precision: 0.9989 - recall: 0.7956 - tn: 2711.0000 - tp: 792808.0000 - val_Brier score: 0.1121 - val_accuracy: 0.8720 - val_auc: 0.9403 - val_cross entropy: 0.3503 - val_fn: 127214.0000 - val_fp: 789.0000 - val_loss: 0.3503 - val_prc: 0.9998 - val_precision: 0.9991 - val_recall: 0.8723 - val_tn: 2878.0000 - va

[0.06077270209789276,
 0.015987329185009003,
 0.9790390133857727,
 0.9990506172180176,
 0.06077270209789276,
 20959.0,
 2.0,
 0.9999966025352478,
 0.9999979734420776,
 0.9789647459983826,
 3623.0,
 975416.0]

In [6]:
nuclei = [pyna.Nucleus("h1"),
          pyna.Nucleus("he4"),
          pyna.Nucleus("c12"),
          pyna.Nucleus("o16"),
          pyna.Nucleus("n14"),
          pyna.Nucleus("ca40")]

reactants = [pyna.Nucleus("c12"), pyna.Nucleus("he4")]

comp = pyna.Composition(nuclei=nuclei)
comp.set_solar_like()

scn_fac = pyna.make_screen_factors(*reactants)

In [7]:
dens, temp = 2.e4, 1.5e9

plasma = pyna.make_plasma_state(temp, dens, comp.get_molar())

In [8]:
chugunov_2009(plasma, scn_fac)

1.0053474914264091

In [9]:
network.predict(temp=temp, dens=dens, comp=comp, nuclei=reactants)

array(True)

In [10]:
T_, D_ = 10**np.linspace(8, 11), 10**np.linspace(4, 8)
T, D = np.meshgrid(T_, D_)

In [11]:
#network.predict(temp=T, dens=D, comp=comp, nuclei=reactants)

In [12]:
x_test = data.x["test"]
hits = x_test[(data.y["test"] == 0).flatten()]
hits

array([[ 9.86040323e+00,  4.93746392e+00,  9.08862473e+00, ...,
         6.56607644e+00,  2.36600246e+00,  4.77081146e+00],
       [ 8.04518199e+00,  6.56226335e+00, -5.56400146e-01, ...,
         1.64825761e+01,  7.65136314e+01,  1.45974167e+02],
       [ 9.90176196e+00,  4.48713677e+00,  5.10196244e+00, ...,
         3.92391698e+00,  3.80509160e+01,  8.23997105e+01],
       ...,
       [ 9.97870813e+00,  6.09682196e+00, -8.93683810e-02, ...,
         8.11545750e+01,  1.00828016e+02,  2.04913659e+02],
       [ 9.28597810e+00,  4.87511388e+00, -4.03114348e-01, ...,
         1.44143294e+02,  7.97685245e+01,  1.58287424e+02],
       [ 9.91091690e+00,  5.91187649e+00, -2.45845303e-01, ...,
         1.46335049e+02,  8.82915907e+01,  1.68039142e+02]])